In [ ]:
from __future__ import print_function
import os.path
import dalmatian as dm
import pandas as pd
import sys
from src.CCLE_postp_function import *
sys.path.insert(0, '../JKBio/')
import Datanalytics as da 
import TerraFunction as terra
from Helper import * 
%load_ext autoreload
%autoreload 2
%load_ext rpy2.ipython
from IPython.display import Image,display
from taigapy import TaigaClient
tc = TaigaClient()
import numpy as np
from sklearn.manifold import TSNE
from sklearn.neighbors import KNeighborsClassifier
from bokeh.plotting import *
from bokeh.models import HoverTool
output_notebook()
from collections import OrderedDict
from gsheets import Sheets
sheets = Sheets.from_files('~/.client_secret.json', '~/.storage.json')
replace = {'T': 'Tumor', 'N': 'Normal', 'm': 'Unknown', 'L': 'Unknown'}

In [ ]:
workspace1="broad-genomics-delivery/Getz_IBM_CellLines_RNASeqData"
workspace2="broad-firecloud-ccle/CCLE_DepMap_RNAseq"
workspace3="broad-genomics-delivery/CCLE_DepMap_RNAseq"
workspace4="broad-genomics-delivery/Cancer_Cell_Line_Factory_CCLF_RNAseq"
workspace5="nci-mimoun-bi-org/CCLF_RNA_2_0"
refworkspace="broad-firecloud-ccle/DepMap_hg38_RNAseq"
source1="ibm"
source2="ccle"
source3="ccle"
source4="cclf"
sample_set_id = "19Q4"
release = sample_set_id

## yet to edit - copied from CN:
# sheeturl = "https://docs.google.com/spreadsheets/d/115TUgA1t_mD32SnWAGpW9OKmJ2W5WYAOs3SuSdedpX4"
# prevfile_gene = 'wes.19Q3.gene'
# prevprevfile_gene ='wes.19Q2.gene'

In [ ]:
wm1 = dm.WorkspaceManager(workspace1)
wm2 = dm.WorkspaceManager(workspace2)
wm3 = dm.WorkspaceManager(workspace3)
wm4 = dm.WorkspaceManager(workspace4)

refwm = dm.WorkspaceManager(refworkspace)

# Generate sample set from new samples

In [ ]:
extract_defaults = {
    'name': 'sample_alias',
    'bai': 'crai_or_bai_path',
    'bam': 'cram_or_bam_path',
    'source': 'source',
    'id': 'individual_alias',
    'participant': 'individual_alias',
    'bait_set': 'bait_set',
    'hs_hs_library_size': 'hs_hs_library_size',
    'hs_het_snp_sensitivity': 'hs_het_snp_sensitivity',
    'hs_mean_bait_coverage': 'hs_mean_bait_coverage',
    'hs_mean_target_coverage': 'hs_mean_target_coverage',
    'hs_on_target_bases': 'hs_on_target_bases',
    'total_reads': 'total_reads',
    'release_date': 'release_date',
    'mean_depth': 'mean_depth',
    'ref_bams':'RNAseq_bam'
}
bam_col_name = {'ref_bams':'RNAseq_bam'}

In [ ]:
# not adding CCLF info (wm4) until 20Q1
newsamples = createDatasetWithNewCellLines(refwm, sample_set_id, wmfroms = [wm1, wm2, wm3], sources=[source1, source2, source3],
                                          gsfolderto='gs://ccle_bams/rna/', match='ACH', participantslicepos=10, 
                                          accept_unknowntypes=True, extract = extract_defaults,
                                          dry_run = True)

# run the pipeline

In [ ]:
## check: is this out-dated? 
## are we using samtofastq_v1-0_BETA instead now?
samtofastq = refwm.get_config("samtofastq_v1-0_BETA_cfg")
samtofastq

In [ ]:
samtofastq['samtofastq_workflow.samtofastq.input_bam_cram']= 'this.WES_bam'
refwm.update_config(samtofastq)
submission_id = refwm.create_submission(samtofastq['name'], sample_set_id,'sample_set',expression='this.samples')

In [ ]:
terra.waitForSubmission(refworkspace, submission_id)

In [ ]:
star = refwm.get_config("star_v1-0_BETA_cfg")
star

In [ ]:
refwm.update_config(star)
submission_id = refwm.create_submission(star['name'], sample_set_id,'sample_set',expression='this.samples')
terra.saveConfigs(refworkspace,'data/'+sample_set_id+'/RNAconfig')

# run the pipeline

In [ ]:
# submission_id = refwm.create_submission("samtofastq_v1-0_BETA", samplesetname,'sample_set',expression='this.samples')
# terra.waitForSubmission(refwm, submission_id)

In [ ]:
submission_id = refwm.create_submission("star_v1-0_BETA_cfg", samplesetname,'sample_set',expression='this.samples')
submission_id_snp = refwm.create_submission("rnaseq-germline-snps-indels", samplesetname,'sample_set',expression='this.samples')
terra.waitForSubmission(refwm, submission_id)

In [ ]:
submission_id1 = refwm.create_submission("rsem_v1-0_BETA_cfg", samplesetname,'sample_set',expression='this.samples')
terra.waitForSubmission(refwm, submission_id1)

In [ ]:
submission_id2 = refwm.create_submission("hg38_STAR_fusion", samplesetname,'sample_set',expression='this.samples')

In [ ]:
updateAllSampleSet('CCLE_'+samplesetname, Allsample_setname='DM19Q2_PATHS_CORRECTED_V2')

In [ ]:
submission_id1 = refwm.create_submission("rsem_aggregate_results_v1-0_BETA_cfg", 'All_samples')

In [ ]:
terra.saveConfigs(refworkspace,'data/'+sample_set_id+'/RNAconfig')

In [ ]:
wait_for_submission(refwm, [submission_id1, submission_id_snp, submission_id2])

## Expression post processing

In [ ]:
res = refwm.get_sample_sets().loc['all_samples']
res

In [ ]:
res = refwm.get_sample_sets().loc['All_samples']
rsem_genes_expected_count = res['rsem_genes_expected_count']
rsem_genes_tpm = res['rsem_genes_tpm']
rsem_transcripts_tpm = res['rsem_transcripts_tpm']
! gsutil cp $rsem_genes_expected_count "temp/" & gsutil cp $rsem_genes_tpm "temp/" & gsutil cp $rsem_transcripts_tpm "temp/"

In [ ]:
%%R
release <- '19Q4'

In [ ]:
%%R
library('taigr')
source('../JKBio/gkugener/RScripts/load_libraries_and_annotations.R')
source('CCLE_postp_function.R')

In [ ]:
%%R
# These files are downloaded from FireCloud/Terra
download_paths <- list(
  tpm_genes='temp/expression.genes.tpm.txt.gz',
  tpm_transcripts='temp/expression.transcript.tpm.txt.gz',
  counts_genes='temp/expression.expectedcount.txt.gz')

In [ ]:
%%R
tpm_transcripts = readTranscripts(download_paths$tpm_transcripts)
counts_genes = readCounts(download_paths$counts_genes)
tpm_genes = readTPM(download_paths$tpm_genes)

In [ ]:
%%R
# Fix the colnames (for cases where there are mixed values (CCLE_name and DepMap_IDs))
colnames(counts_genes) %<>% renameFunction(.)
colnames(tpm_genes) %<>% renameFunction(.)
colnames(tpm_transcripts) %<>% renameFunction(.)

### data exploration and QC

In [ ]:
%%R
tpm_genes[,'ACH-001767']

In [ ]:
%%R 
# Quick QC
# We are looking for samples with a worrying amount of zeros
zero_threshold <- 39000
number_zeros <- apply(tpm_genes[,3:ncol(tpm_genes)], 2, FUN = function(x) length(x[x == 0]))
nzdf <- data.frame(CL=names(number_zeros), nz=number_zeros, stringsAsFactors = F)

In [ ]:
%%R
number_zeros <- number_zeros[order(-number_zeros)]
number_zeros <- number_zeros[number_zeros < zero_threshold]
pass <- number_zeros %>% names()

# These samples failed
failed <- setdiff(colnames(tpm_genes), pass) %>% .[!(. %in% c('gene_id', 'transcript_id(s)'))]

counts_genes %<>% dplyr::select(c("gene_id","transcript_id(s)", pass))
tpm_genes %<>% dplyr::select(c("gene_id","transcript_id(s)", pass))
tpm_transcripts %<>% dplyr::select(c("transcript_id", "gene_id", pass))

In [ ]:
%%R
failed

In [ ]:
%%R
# Plot of the samples that fail
plot <- ggplot(nzdf, aes(nz)) +
  geom_histogram(bins = 100, color='black', fill='white') +
  geom_vline(xintercept = zero_threshold, linetype=2) +
  geom_label_repel(data = nzdf %>% filter(nz > zero_threshold), aes(x=nz, y=0, label=CL), size=5, fill=rgb(1,1,1,0.5))

ggsave(plot, filename ='temp/ggplot.png', width=20, height = 20)

In [ ]:
display(Image(filename='temp/ggplot.png'))

In [ ]:
%%R
write.table(
  counts_genes, 
  file = paste0('temp/expression.', release,'.counts.tsv'), 
  sep = '\t', row.names = F, quote = F)
write.table(
  tpm_genes, 
  file = paste0('temp/expression.', release,'genes.tsv'), 
  sep = '\t', row.names = F, quote = F)
write.table(
  tpm_transcripts, 
  file = paste0('temp/expression.', release,'.transcripts.tsv'),
  sep = '\t', row.names = F, quote = F)

# Validation

In [ ]:
counts_genes = pd.read_csv('temp/expression.'+ release + '.counts.tsv', sep='\t')
tpm_genes = pd.read_csv('temp/expression.'+ release + '.tpm.tsv', sep='\t')
tpm_transcripts = pd.read_csv('temp/expression.'+ release + '.transcripts.tsv', sep='\t')

In [ ]:
prevcounts = tc.get(name='depmap-expression-87f8', file="expression."+prevname+'.counts')
prevtpm = tc.get(name='depmap-expression-87f8', file="expression."+prevname+'.tpm')
prevtranscripts = tc.get(name='depmap-expression-87f8', file="expression."+prevname+'.transcripts')

In [ ]:
new1 = set(counts_genes.columns.values.tolist())
prev = set(prev.columns.values.tolist())

In [ ]:
new3 = set(tpm_transcripts.columns.values.tolist())
new2 = set(tpm_genes.columns.values.tolist())

In [ ]:
print(len(prev), len(prev & new2), len(new1), len(new1 & new2 & new3), new2-prev, prev-new2)

In [ ]:
metadata = tc.get(name='depmap-a0ab', version=24, file='sample_info')

In [ ]:
metadata

In [ ]:
# finding train and test set
trainame = [val for val in new1&prev if val[:3] == 'ACH']
testname = [val for val in new1-prev if val[:3] == 'ACH']

#looking at the 2000 most variable genes in the two sets
genetolookfor = 2000
gene_var = counts_genes[trainame].var(1).values
print(len(gene_var))
sorting = np.argsort(gene_var)[-genetolookfor:]

In [ ]:
unregistered = set(testname) - set(metadata["DepMap_ID"].values.tolist())
unregistered

In [ ]:
len(counts_genes['ACH-001767']) - np.count_nonzero(counts_genes['ACH-001767'].values)

In [ ]:
# creating and reodering train and test sets
traindata = counts_genes[set(trainame)-unregistered].values[sorting].T
trainlabels = [metadata[metadata["DepMap_ID"]==val]["disease"].values[0] for val in counts_genes[set(trainame)-unregistered].columns.tolist() if val not in unregistered]

testdata = counts_genes[set(testname)-unregistered].values[sorting].T
testlabels = [metadata[metadata["DepMap_ID"]==val]["disease"].values[0] for val in counts_genes[set(testname)-unregistered].columns.tolist() if val not in unregistered]

In [ ]:
traindata.shape

In [ ]:
# learn KNN classifier to the metadata diseases
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(traindata, trainlabels) 
predicted = neigh.predict(testdata)

In [ ]:
predicted[1]

In [ ]:
labels = trainlabels + testlabels
colors=[0]*len(trainlabels)
colors.extend([1,2,2,2,2,1,2,2,2,1,2])
data = np.vstack([traindata,testdata])

In [ ]:
# plot them with TSNE, highlight the points that failed and show colors for diseases
dimred = TSNE(2,10).fit_transform(data)

In [ ]:
scatter(dimred, labels=labels,colors=colors, radi=1.9)

## Save files for taiga

In [ ]:
tc.update_dataset(dataset_permaname="depmap-expression-87f8",
                 upload_file_path_dict={'temp/expression.'+release+'.transcripts.tsv': 'TableTSV',
                                       'temp/expression.'+release+'.genes.tsv': 'TableTSV',
                                       'temp/expression.'+release+'.counts.tsv': 'TableTSV'},
                  dataset_description=
"""
# RNAseq

Combined segment and gene-level CN calls from Broad WES, Sanger WES, and Broad SNP. Relative CN, not log2 transformed.

PORTAL TEAM SHOULD NOT USE THIS: There are lines here that should not make it even to internal. Must use subsetted dataset instead. These data will not make it on the portal starting 19Q1. With the DMC portal, there is new cell line release prioritization as to which lines can be included, so a new taiga dataset will be created containing CN for the portal.

version 1-8: guillaume releases
version 9: 19Q3 release
version 10:  adding missing samples in Terra merge files
version 11: 19Q4 new release.

## ** Internal 19Q4****

Adding 17 new cell lines, 3 reprioritized cell lines. log2(COPY RATIO+1). 
Some cells lines have been flagged as:

 - having bad looking copy ration plots = 
 - Genes having a similar CN value accross all []

transcriptoins (Transcripts rpkm):

genes (gene rpkm):
__Rows__:
__Columns__:
Counts (gene counts):
__Rows__:
__Columns__:
Gene level CN data:
__Rows__:
__Columns__:
 DepMap cell line IDs
 gene names in the format HGNC\_symbol (Entrez\_ID)
DepMap\_ID, Chromosome, Start, End, Num\_Probes, Segment\_Mean
""")

## Fusion post processing

In [ ]:
terra.wait_for_submission(submission_id2)

In [ ]:
submission_id2 = refwm.create_submission("Aggregate_Fusion_Calls", 'All_samples')
terra.wait_for_submission(submission_id2)

In [ ]:
refwm.get_sample_sets().loc['All_samples']['fusions_star']

In [ ]:
! gsutil cp $aggregated "temp/expression.fusion.tsv"

### Overview

This document contains the code used to generate the unfiltered and filtered versions of the fusion datasets for the release. The bottom of the document also contains some comparisons between the release fusion dataset, CCLE2 fusion calls, and the translocation data from CCLE2.

In [ ]:
%%R
source('../JKBio/gkugener/RScripts/load_libraries_and_annotations.R')
source("CCLE_postp_function.R")

## Generate filtered fusion table

Release: `r release`

We want to apply filters to the fusion table to reduce the number of artifacts in the dataset. Specifically, we filter the following:

* Remove fusions involving mitochondrial chromosomes, or HLA genes, or immunoglobulin genes
* Remove red herring fusions (from STAR-Fusion annotations column)
* Remove recurrent in CCLE (>= 25 samples)
* Remove fusion with (SpliceType=" INCL_NON_REF_SPLICE" and LargeAnchorSupport="No" and FFPM < 0.1)
* Remove fusions with FFPM < 0.05 (STAR-Fusion suggests using 0.1, but looking at the translocation data, this looks like it might be too aggressive)

In [ ]:
%%R
unfiltered_fusions <- readFusions('temp/expression.fusion.tsv')
filtered_fusions <- filterFusions(unfiltered_fusions)

In [ ]:
%%R
# Save the files (to be uploaded to taiga)
write.table(
  unfiltered_fusions,
  file = paste0('temp/fusions.',release, '.unfiltered.tsv'),
  sep = '\t', quote = F, row.names = F
)
write.table(
  filtered_fusions,
  file = paste0('temp/fusions.', release, '.filtered.tsv'),
  sep = '\t', quote = F, row.names = F
)

# Validation

## Validation Protocol:

to validate fusions, one should be able to list all cells with known fusions (i.e. elwing sarcoma) and check for each new cell in this set of knownfusioncells, if the fusion is present or not. and validate the fusion quality this way.

In [ ]:
# check that all cells lines are present on fusion unfiltered
df = pd.read_csv('temp/fusions.'+release+'.unfiltered.tsv', sep='\t')
new = set([i.split('_')[1][:10] for i in list(set(df["DepMap_ID"].tolist()))])
print(prev - new, new-prev)

In [ ]:
# remove fusions from the same samples as for that failed expression threshold


In [ ]:
unfiltered = pd.read_csv('temp/fusions.'+release+'.unfiltered.tsv', sep='\t')
filtered = pd.read_csv('temp/fusions.'+ release+ '.filtered.tsv',sep='\t')

In [ ]:
unfiltered["DepMap_ID"] = ['ACH' + i.split('_ACH')[1].split('.')[0] for i in unfiltered["DepMap_ID"]]

In [ ]:
filtered["DepMap_ID"] = ['ACH' + i.split('_ACH')[1].split('.')[0] for i in filtered["DepMap_ID"]]

In [ ]:
prevfiltered = tc.get(name='depmap-fusions-7990', file="fusions."+prevname+".filtered")
prevunfiltered = tc.get(name='depmap-fusions-7990', file='fusions.'+prevname+'.unfiltered')

In [ ]:
filtered

In [ ]:
q1filtered

In [ ]:
filtered = pd.concat([filtered,q1filtered[q1filtered['DepMap_ID']=='ACH-000309']], sort=False)
unfiltered = pd.concat([unfiltered,q1unfiltered[q1unfiltered['DepMap_ID']=='ACH-000309']], sort=False)

In [ ]:
unfiltered.to_csv('temp/fusions.'+release+'.unfiltered.tsv', sep='\t')
filtered.to_csv('temp/fusions.'+ release+ '.filtered.tsv',sep='\t')

# Uploading to Taiga

In [ ]:
tc.update_dataset(dataset_permaname="depmap-fusions-7990",
                     upload_file_path_dict={'temp/fusions.'+release+'.filtered.tsv': 'TableTSV',
                                        'temp/fusions.'+release+'.unfiltered.tsv': 'TableTSV'},
                 dataset_description="""
# Fusions

filtered and unfiltered fusion files from Broad RNAseq data mapped to hg38

PORTAL TEAM SHOULD NOT USE THIS: There are lines here that should not make it even to internal. Must use subsetted dataset instead. These data will not make it on the portal starting 19Q1. With the DMC portal, there is new cell line release prioritization as to which lines can be included, so a new taiga dataset will be created containing CN for the portal.

version 1-4: guillaume releases
version 5: 19Q3 release
version 6:  adding missing samples in Terra merge files
version 7: 19Q4 new release.

## ** Internal 19Q4****

Adding 17 new cell lines, 3 reprioritized cell lines. log2(COPY RATIO+1). 
Some cells lines have been flagged as:

 - having bad looking copy ration plots = 
 - Genes having a similar CN value accross all []

transcriptoins (Transcripts rpkm):

genes (gene rpkm):
__Rows__:
__Columns__:
Counts (gene counts):
__Rows__:
__Columns__:
Gene level CN data:
__Rows__:
__Columns__:
 DepMap cell line IDs
 gene names in the format HGNC\_symbol (Entrez\_ID)
DepMap\_ID, Chromosome, Start, End, Num\_Probes, Segment\_Mean
 """)

# [Additional] IF want to merge here instead of on Terra:

In [ ]:
newsamples

In [ ]:
prevsamplesets = ['CCLE_19Q3interim',sample_set_id]
samples = []
for i in prevsamplesets:
    samples.extend(refwm.get_sample_sets().loc[i].samples)
res = []
terrasamp = refwm.get_samples()
for i, sample in enumerate(samples):
    res.append(terrasamp.loc[sample])
    genes_fusion = res[i]['fusion_predictions_abridged']
    rsem_genes_transcripts = res[i]['rsem_isoforms']
    rsem_genes_expected_count = res[i]['rsem_genes']
    ! gsutil cp $rsem_genes_expected_count 'temp/' && gsutil cp $rsem_genes_transcripts 'temp/' && gsutil cp $genes_fusion 'temp/'

In [ ]:
mainres = refwm.get_sample_sets().loc['DM19Q2_PATHS_CORRECTED_V2']
maingenes_fusion = mainres['fusions_star']
mainrsem_genes_tpm = mainres['rsem_genes_tpm']
mainrsem_genes_transcripts = mainres['rsem_transcripts_tpm']
mainrsem_genes_expected_count = mainres['rsem_genes_expected_count']
! gsutil cp $mainrsem_genes_expected_count "temp/" && gsutil cp $mainrsem_genes_transcripts "temp/" && gsutil cp $maingenes_fusion "temp/expression.fusion.tsv" && gsutil cp $mainrsem_genes_tpm "temp/"

In [ ]:
mainres['rsem_genes_expected_count']

In [ ]:
addSamplesRSEMToMain(res,mainres)

In [ ]:
genes_fusion = ['temp/'+val['fusion_predictions_abridged'].split('/')[-1] for val in res]
addToMainFusion(genes_fusion,'temp/expression.fusion.tsv')